In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gmaps
from config import gkey

In [2]:
data = pd.read_csv("aac_intakes_outcomes_address_income.csv", encoding="utf-8", low_memory=False)

In [3]:
data.head()

,age_upon_outcome,animal_id_outcome,date_of_birth,outcome_subtype,outcome_type,sex_upon_outcome,age_upon_outcome_(days),age_upon_outcome_(years),age_upon_outcome_age_group,outcome_datetime,...,time_in_shelter,time_in_shelter_days,age_upon_intake_number,age_upon_intake_lentime,found_location_street,zipcode,income,population,Lat,Lon
0,10 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,3650,10.0,"(7.5, 10.0]",2017-12-07 14:07:00,...,0 days 14:07:00.000000000,0.588194,10,years,Colony Creek And Hunters Trace,78758,"47,244","44,072",30.387069,-97.703656
1,7 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,2555,7.0,"(5.0, 7.5]",2014-12-20 16:35:00,...,1 days 06:14:00.000000000,1.259722,7,years,8700 Research Blvd,78757,"62,992","21,310",30.356821,-97.730807
2,6 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,2190,6.0,"(5.0, 7.5]",2014-03-08 17:10:00,...,1 days 02:44:00.000000000,1.113889,6,years,8700 Research,78757,"62,992","21,310",30.356821,-97.730807
3,10 years,A047759,2004-04-02 00:00:00,Partner,Transfer,Neutered Male,3650,10.0,"(7.5, 10.0]",2014-04-07 15:12:00,...,4 days 23:17:00.000000000,4.970139,10,years,NaN,NaN,NaN,NaN,NaN,NaN
4,16 years,A134067,1997-10-16 00:00:00,NaN,Return to Owner,Neutered Male,5840,16.0,"(15.0, 17.5]",2013-11-16 11:54:00,...,0 days 02:52:00.000000000,0.119444,16,years,12034 Research Blvd,78759,"74,379","38,891",30.401356,-97.752535


In [4]:
geo_factors = data[["animal_id_outcome", "zipcode", "animal_type"]]
geo_factors.head()

,animal_id_outcome,zipcode,animal_type
0,A006100,78758,Dog
1,A006100,78757,Dog
2,A006100,78757,Dog
3,A047759,NaN,Dog
4,A134067,78759,Dog


In [5]:
geo_factors_clean = geo_factors.loc[(geo_factors["zipcode"] != None) & (geo_factors["zipcode"].str.isnumeric())]
geo_factors_groupZC = geo_factors_clean.groupby(["zipcode", "animal_type"])

In [6]:
geo_factors_groupZC = geo_factors_groupZC.count()
geo_factors_groupZC.head()

,,animal_id_outcome
zipcode,animal_type,
01223,Dog,1
01602,Dog,4
01604,Dog,1
01810,Dog,1
01960,Cat,1


In [7]:
geo_factors_groupZC.sort_values(by=["animal_type", "animal_id_outcome", "zipcode"], ascending=False, inplace=True)

In [8]:
geo_factors_groupZC = geo_factors_groupZC.reset_index()

In [9]:
geo_factors_groupZC.head()

,zipcode,animal_type,animal_id_outcome
0,78701,Other,408
1,78704,Other,317
2,78753,Other,187
3,78745,Other,176
4,78744,Other,175


In [10]:
zipcode_geo = data[["zipcode", "Lat", "Lon"]]
zipcode_geo = zipcode_geo.drop_duplicates()
zipcode_geo.head()

,zipcode,Lat,Lon
0,78758,30.387069,-97.703656
1,78757,30.356821,-97.730807
3,NaN,NaN,NaN
4,78759,30.401356,-97.752535
6,78751,30.305571,-97.725376


In [11]:
geo_data = pd.merge(geo_factors_groupZC, zipcode_geo, how="left", on="zipcode")

In [12]:
geo_data.head()

,zipcode,animal_type,animal_id_outcome,Lat,Lon
0,78701,Other,408,30.272921,-97.744386
1,78704,Other,317,30.245673,-97.768836
2,78753,Other,187,30.388987,-97.671089
3,78745,Other,176,30.215387,-97.796012
4,78744,Other,175,30.196311,-97.730807


In [13]:
gmaps.configure(api_key=gkey)

In [14]:
def filterByAnimalTypeLocation(dataset, animalType, top):
    data = dataset.loc[dataset["animal_type"] == animalType]
    if top==0:
            return data[["Lat", "Lon"]]
    return data[["Lat", "Lon"]][0:top]
def filterByAnimalTypeRaiting(dataset, animalType, top):
    data = dataset.loc[dataset["animal_type"] == animalType]
    if top==0:
        return data["animal_id_outcome"].astype(float)
    return data["animal_id_outcome"].astype(float)[0:top]

In [18]:
fig = gmaps.figure()
locations = filterByAnimalTypeLocation(geo_data, "Dog", 0)
rating = filterByAnimalTypeRaiting(geo_data, "Dog", 0)
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=True, max_intensity=10,
                                 point_radius=None)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
fig = gmaps.figure()
locations = filterByAnimalTypeLocation(geo_data, "Cat", 0)
rating = filterByAnimalTypeRaiting(geo_data, "Cat", 0)
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=True, max_intensity=10,
                                 point_radius=None)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [83]:
fig = gmaps.figure()
locations = filterByAnimalTypeLocation(geo_data, "Bird", 0)
rating = filterByAnimalTypeRaiting(geo_data, "Bird", 0)
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=True, max_intensity=10,
                                 point_radius=None)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [84]:
fig = gmaps.figure()
locations = filterByAnimalTypeLocation(geo_data, "Other", 0)
rating = filterByAnimalTypeRaiting(geo_data, "Other", 0)
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=True, max_intensity=10,
                                 point_radius=None)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))